In [ ]:
from os import listdir
from os.path import isfile, join
import string

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
my_path = '/content/drive/My Drive/Colab Notebooks/20newsgroup'

#creating a list of folder names to make valid pathnames later
folders = [f for f in listdir(my_path)]

In [ ]:
folders

['alt.atheism', 'sci.space']

In [ ]:
#creating a 2D list to store list of all files in different folders

files = []
for folder_name in folders:
    folder_path = join(my_path, folder_name)
    files.append([f for f in listdir(folder_path)])

In [ ]:
#checking total no. of files gathered

sum(len(files[i]) for i in range(2))

2000

In [ ]:
#creating a list of pathnames of all the documents
#this would serve to split our dataset into train & test later without any bias

pathname_list = []
for fo in range(len(folders)):
    for fi in files[fo]:
        pathname_list.append(join(my_path, join(folders[fo], fi)))

In [ ]:
len(pathname_list)

2000

In [ ]:

#making an array containing the classes each of the documents belong to

Y = []
for folder_name in folders:
    folder_path = join(my_path, folder_name)
    num_of_files= len(listdir(folder_path))
    for i in range(num_of_files):
        Y.append(folder_name)

In [ ]:
len(Y)

2000

Spliting data to train and test

In [ ]:
from sklearn.model_selection import train_test_split
doc_train, doc_test, Y_train, Y_test = train_test_split(pathname_list, Y, random_state=0, test_size=0.25)

functions for word extraction from documents

In [ ]:
stopwords = ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at',
 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 
 'can', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during',
 'each', 'few', 'for', 'from', 'further', 
 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's",
 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's",
 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself',
 "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself',
 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours' 'ourselves', 'out', 'over', 'own',
 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 
 'than', 'that',"that's", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", 
 "they'll", "they're", "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 
 'was', "wasn't", 'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
 "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's",'will', 'with', "won't", 'would', "wouldn't", 
 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves', 
 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'hundred', 'thousand', '1st', '2nd', '3rd',
 '4th', '5th', '6th', '7th', '8th', '9th', '10th']

In [ ]:
#function to preprocess the words list to remove punctuations

def preprocess(words):
    #we'll make use of python's translate function,that maps one set of characters to another
    #we create an empty mapping table, the third argument allows us to list all of the characters 
    #to remove during the translation process
    
    #first we will try to filter out some  unnecessary data like tabs
    table = str.maketrans('', '', '\t')
    words = [word.translate(table) for word in words]
    
    punctuations = (string.punctuation).replace("'", "") 
    # the character: ' appears in a lot of stopwords and changes meaning of words if removed
    #hence it is removed from the list of symbols that are to be discarded from the documents
    trans_table = str.maketrans('', '', punctuations)
    stripped_words = [word.translate(trans_table) for word in words]
    
    #some white spaces may be added to the list of words, due to the translate function & nature of our documents
    #we remove them below
    words = [str for str in stripped_words if str]
    
    #some words are quoted in the documents & as we have not removed ' to maintain the integrity of some stopwords
    #we try to unquote such words below
    p_words = []
    for word in words:
        if (word[0] and word[len(word)-1] == "'"):
            word = word[1:len(word)-1]
        elif(word[0] == "'"):
            word = word[1:len(word)]
        else:
            word = word
        p_words.append(word)
    
    words = p_words.copy()
        
    #we will also remove just-numeric strings as they do not have any significant meaning in text classification
    words = [word for word in words if not word.isdigit()]
    
    #we will also remove single character strings
    words = [word for word in words if not len(word) == 1]
    
    #after removal of so many characters it may happen that some strings have become blank, we remove those
    words = [str for str in words if str]
    
    #we also normalize the cases of our words
    words = [word.lower() for word in words]
    
    #we try to remove words with only 2 characters
    words = [word for word in words if len(word) > 2]
    
    return words

In [ ]:
#function to remove stopwords

def remove_stopwords(words):
    words = [word for word in words if not word in stopwords]
    return words

In [ ]:
#function to convert a sentence into list of words

def tokenize_sentence(line):
    words = line[0:len(line)-1].strip().split(" ")
    words = preprocess(words)
    words = remove_stopwords(words)
    
    return words

In [ ]:
#function to remove metadata

def remove_metadata(lines):
    for i in range(len(lines)):
        if(lines[i] == '\n'):
            start = i+1
            break
    new_lines = lines[start:]
    return new_lines

In [ ]:
#function to convert a document into list of words

def tokenize(path):
    #load document as a list of lines
    f = open(path, 'r',encoding="utf8", errors='ignore')
    text_lines = f.readlines()
    
    #removing the meta-data at the top of each document
    text_lines = remove_metadata(text_lines)
    
    #initiazing an array to hold all the words in a document
    doc_words = []
    
    #traverse over all the lines and tokenize each one with the help of helper function: tokenize_sentence
    for line in text_lines:
        doc_words.append(tokenize_sentence(line))

    return doc_words

In [ ]:
#a simple helper function to convert a 2D array to 1D, without using numpy

def flatten(list):
    new_list = []
    for i in list:
        for j in i:
            new_list.append(j)
    return new_list

using the above functions on actual documents


In [ ]:
len(folders)

2

In [ ]:
list_of_words = []

for document in doc_train:
        print(document)
        list_of_words.append(flatten(tokenize(document)))

/content/drive/My Drive/Colab Notebooks/20newsgroup/sci.space/59848
/content/drive/My Drive/Colab Notebooks/20newsgroup/alt.atheism/54255
/content/drive/My Drive/Colab Notebooks/20newsgroup/sci.space/61358
/content/drive/My Drive/Colab Notebooks/20newsgroup/alt.atheism/53322
/content/drive/My Drive/Colab Notebooks/20newsgroup/alt.atheism/54191
/content/drive/My Drive/Colab Notebooks/20newsgroup/sci.space/60812
/content/drive/My Drive/Colab Notebooks/20newsgroup/sci.space/60248
/content/drive/My Drive/Colab Notebooks/20newsgroup/alt.atheism/54250
/content/drive/My Drive/Colab Notebooks/20newsgroup/alt.atheism/53474
/content/drive/My Drive/Colab Notebooks/20newsgroup/alt.atheism/53126
/content/drive/My Drive/Colab Notebooks/20newsgroup/alt.atheism/53572
/content/drive/My Drive/Colab Notebooks/20newsgroup/sci.space/60216
/content/drive/My Drive/Colab Notebooks/20newsgroup/alt.atheism/53055
/content/drive/My Drive/Colab Notebooks/20newsgroup/alt.atheism/53208
/content/drive/My Drive/Colab 

In [ ]:
len(list_of_words)

1500

In [ ]:
len(flatten(list_of_words))

215797

from above lengths we observe that the code has been designed in as such a way that the 2D list: list_of_words contains the vocabulary of each document file in the each of its rows, and collectively contains all the words we extract from the 20_newsgroups folder


In [ ]:
n = 5000
features = wrds[0:n]
print(features)

['writes', 'article', 'space', 'people', 'just', 'like', 'god', 'think', 'know', 'say', 'also', 'time', 'see', 'get', 'even', 'may', 'many', 'way', 'system', 'much', 'good', 'believe', 'make', 'now', 'well', 'something', 'first', 'science', 'said', 'new', 'things', 'launch', 'since', 'right', 'point', 'must', 'earth', 'religion', 'still', 'use', 'nasa', 'fact', 'better', 'moral', 'really', 'evidence', 'orbit', 'objective', 'might', 'world', 'find', 'read', 'thing', 'years', 'different', 'jesus', 'made', 'data', 'want', 'shuttle', 'someone', 'question', 'part', 'take', 'mission', 'work', 'used', 'life', 'need', 'anyone', 'book', 'mean', 'atheists', 'never', 'true', 'information', 'course', 'idea', 'back', 'atheism', 'real', 'rather', 'anything', 'look', 'possible', 'example', 'however', 'morality', 'nothing', 'sure', 'another', 'seems', 'etc', 'actually', 'satellite', 'enough', 'reason', 'wrong', 'without', 'program', 'going', 'long', 'every', 'please', 'others', 'little', 'least', 'bil

In [ ]:

#creating a dictionary that contains each document's vocabulary and ocurence of each word of the vocabulary 

dictionary = {}
doc_num = 1
for doc_words in list_of_words:
    #print(doc_words)
    np_doc_words = np.asarray(doc_words)
    w, c = np.unique(np_doc_words, return_counts=True)
    dictionary[doc_num] = {}
    for i in range(len(w)):
        dictionary[doc_num][w[i]] = c[i]
    doc_num = doc_num + 1

In [ ]:
dictionary.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 22

In [ ]:
#now we make a 2D array having the frequency of each word of our feature set in each individual documents

X_train = []
for k in dictionary.keys():
    row = []
    for f in features:
        if(f in dictionary[k].keys()):
            #if word f is present in the dictionary of the document as a key, its value is copied
            #this gives us no. of occurences
            row.append(dictionary[k][f]) 
        else:
            #if not present, the no. of occurences is zero
            row.append(0)
    X_train.append(row)

In [ ]:
#we convert the X and Y into np array for concatenation and conversion into dataframe

X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)

In [ ]:
print(len(X_train))
print(len(Y_train))

1500
1500


we'll make our test data by performing the same operations as we did for train data


In [ ]:
list_of_words_test = []

for document in doc_test:
        list_of_words_test.append(flatten(tokenize(document)))

In [ ]:
dictionary_test = {}
doc_num = 1
for doc_words in list_of_words_test:
    #print(doc_words)
    np_doc_words = np.asarray(doc_words)
    w, c = np.unique(np_doc_words, return_counts=True)
    dictionary_test[doc_num] = {}
    for i in range(len(w)):
        dictionary_test[doc_num][w[i]] = c[i]
    doc_num = doc_num + 1

In [ ]:

#now we make a 2D array having the frequency of each word of our feature set in each individual documents

X_test = []
for k in dictionary_test.keys():
    row = []
    for f in features:
        if(f in dictionary_test[k].keys()):
            #if word f is present in the dictionary of the document as a key, its value is copied
            #this gives us no. of occurences
            row.append(dictionary_test[k][f]) 
        else:
            #if not present, the no. of occurences is zero
            row.append(0)
    X_test.append(row)

In [ ]:
X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)

In [ ]:
print(len(X_test))
print(len(Y_test))

500
500



Text Classification

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
def createmodel():
    model = Sequential()
    model.add(Embedding(2000, 120,input_length = X_train.shape[1]))
    model.add(LSTM(196, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(Y_train)
y = to_categorical(integer_encoded)

In [ ]:
model = createmodel()
model.fit(X_train, y, epochs = 5, batch_size=32, verbose = 2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
 - 735s - loss: 0.6946 - accuracy: 0.4967
Epoch 2/5
 - 728s - loss: 0.6956 - accuracy: 0.4867
Epoch 3/5
 - 727s - loss: 0.6932 - accuracy: 0.5053
Epoch 4/5
 - 749s - loss: 0.6940 - accuracy: 0.5020
Epoch 5/5
 - 811s - loss: 0.6937 - accuracy: 0.4967


In [ ]:
labelencoder = LabelEncoder()
integer_encoded1 = labelencoder.fit_transform(Y_test)
y1 = to_categorical(integer_encoded1)

In [ ]:
score,acc = model.evaluate(X_test,y1,verbose=2,batch_size=32)
print(score)
print(acc)
print(model.metrics_names)

0.6929565315246582
0.5099999904632568
['loss', 'accuracy']
